In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from google.colab import files
path_to_file = list(files.upload().keys())[0]

Saving sherlock-holm.es_stories_plain-text_advs.txt to sherlock-holm.es_stories_plain-text_advs.txt


In [ ]:
#Read the file
with open(path_to_file, 'r',encoding = 'utf-8') as file:
  text = file.read()

In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [ ]:
# Now let's tokenize the text to create a sequence of words:

tokenizer = Tokenizer()

tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

In the above code , the text is tokenized , which means it is divided into individual word or tokens. The 'Tokenizer' object is created ,
 which will handle the tokenization process. The 'fit_on_texts' methods of the tokenizer is called, passing the 'text' as input .
 This method analyzes the text and builds  a vocabulary of unique words , assigning each word a numerical index. The 'total_words' variable is then assigned the value of the length of the word index plus one ,
 representing the total number of distinct words in the text.

In [ ]:
# Now let's create input-output pairs by spltting the text into sequences of tokens and forming n-grams from the sequences:
input_sequences = []
for line in text.split('\n'):
  token_list = tokenizer.texts_to_sequences([line])[0]
  for i in range(1, len(token_list)):
    n_gram_sequence = token_list[:i+1]
    input_sequences.append(n_gram_sequence)




In the above code, the text data is split inti lines using the '\n' character as a delimiter. For each line in the text, the 'texts_to_sequences' method of the tokenizer is used to convert the line into a sequence of numerical tokens based on the previously created vocabulary. The resulting token list is then iterated over using a for loop. For each iteration , a subsequence, or n-gram, of tokens is extracted , ranging from the beginning of the token list up to the current index 'i'.

This n-gram sequence represents the input context, with the last token being the target or predicted word. This n-gram sequence is then appended to the "input_sequences" list. This process is repeated for all lines in the text, generating multiple input-output sequences that will be used for training the next word prediction model.


In [ ]:
# Now let's pad the input sequences to have equal length:
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen = max_sequence_len, padding = 'pre'))


In the above code, the input sequences are padded to ensure all sequences have the same length.
 The variable 'max_sequence_len' is assignwd the maximum length among all the input sequences.
 The 'pad_sequences' function is used to pad or truncate the input sequences to match this maximum length.

The 'pad_sequences' function takes the input_sequences list, sets the maximum length to 'max_sequence_len', and specifies that the padding should be added at the beginning of each sequence using the 'padding=pre' argument. Fnally, the input sequences are converted into a numpy array to facilliate further processing.
Now let's split the sequences into input and output:

In [ ]:
X = input_sequences[:,:-1]
y = input_sequences[:,-1]

In the above code, the input sequences are split into two arrays, 'X' and 'y', to create the input and output for training the next word prediction model.
The 'X' array is assigned the values of all in the 'input_sequences' array except for the last column.
It means that 'X' contains all the tokens in each sequence except for the last one , representing the input context.

Now let's convert the output to one-hot encode vectors:

In [ ]:
y = np.array(tf.keras.utils.to_categorical(y, num_classes = total_words))

In the above code, we are converting the output array into a suitable format for training a model, where each target word is represented as a binary vector.

Now let's build a neural network architecture to train the model:

In [ ]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length = max_sequence_len-1))
model.add(LSTM(150))
model.add(Dense(total_words, activation = 'softmax'))
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 17, 100)           820000    
                                                                 
 lstm_1 (LSTM)               (None, 150)               150600    
                                                                 
 dense_1 (Dense)             (None, 8200)              1238200   
                                                                 
Total params: 2208800 (8.43 MB)
Trainable params: 2208800 (8.43 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


(1). 'total_words', which represents the total number of distinct words in the vocabulary;

(2). '100', which denotes the dimensionality of the word embediings;

(3). and 'input_lengths', which specifies the length of the input sequences.

The next layer added is the 'LSTM' layer , a type of recurrent neural network (RNN) layer designed for capturing sequential dependencies in the data. It has 150 units, which means it will learn 150 internal representations or memory cells.

Finally, the 'Dense' layer is added, which is a fully connected layer that produces the output predictions. It has 'total_words' units and uses the 'softmax' activation function to convert the predicted scores into probabilities, indicating the likelihood of each word being the next one in the sequence.

Now let's compile and train the model:

In [ ]:

model.compile(loss = 'categorical_crossentropy', optimizer = 'adam' , metrics = ['accuracy'])
model.fit(X, y, epochs = 100, verbose = 1)

Epoch 1/100
3010/3010 [==============================] - 270s 89ms/step - loss: 6.2433 - accuracy: 0.0759
Epoch 2/100
3010/3010 [==============================] - 232s 77ms/step - loss: 5.5218 - accuracy: 0.1231
Epoch 3/100
3010/3010 [==============================] - 250s 83ms/step - loss: 5.1245 - accuracy: 0.1475
Epoch 4/100
3010/3010 [==============================] - 219s 73ms/step - loss: 4.7911 - accuracy: 0.1652
Epoch 5/100
3010/3010 [==============================] - 219s 73ms/step - loss: 4.4854 - accuracy: 0.1823
Epoch 6/100
3010/3010 [==============================] - 218s 73ms/step - loss: 4.1993 - accuracy: 0.2017
Epoch 7/100
3010/3010 [==============================] - 219s 73ms/step - loss: 3.9301 - accuracy: 0.2267
Epoch 8/100
3010/3010 [==============================] - 217s 72ms/step - loss: 3.6753 - accuracy: 0.2547
Epoch 9/100
3010/3010 [==============================] - 220s 73ms/step - loss: 3.4322 - accuracy: 0.2897
Epoch 10/100
3010/3010 [======================

In [ ]:
# seed_text = "I will "
seed_text = "Are you"
next_words = 3

for _ in range(next_words):
  token_list = tokenizer.texts_to_sequences([seed_text])[0]
  token_list = pad_sequences([token_list], maxlen = max_sequence_len - 1, padding = 'pre')
  predicted = np.argmax(model.predict(token_list), axis = -1)
  output_word = ""
  for word, index in tokenizer.word_index.items():
    if index == predicted:
      output_word = word
      break
  seed_text += " " + output_word

print(seed_text)

The above code generates the next word predictions based on a given seed text . The 'seed_text' variable holds the initial text. The 'next_words' variable detrmines the number of predictions to be generated. Inside the for loop, the 'seed_text' is converted into a sequence of tokens using the tokenizer. The token sequence is padded to match the maximum sequence length .

The model predicts the next word by calling the 'predict' method on the model with the padded token sequence. The predicted word is obtained by finding the word  with the highest probability score  using 'np.argmax'. Then, the predicted word is appended ro the 'seed_text', and the process is repeated for the desired number of